In [206]:
import pandas as pd
import os
import glob
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import math
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MaxAbsScaler


In [201]:
files = [file for file in os.listdir('./parametros')]

param = pd.DataFrame()
for file in files:
  df = pd.read_csv('./parametros/'+file)
  param = pd.concat([param, df], axis= 0)

In [204]:
from sklearn.model_selection import train_test_split
param = param.sample(frac=1, random_state=42).reset_index(drop=True)
train_df, test_df = train_test_split(param, test_size=0.2, random_state=42)

# Ahora invertimos la red, de input será Fracción vol, espesor y radio. Output longitud de onda e Índice de absorción

Normalizacion

In [207]:
scaler = MaxAbsScaler()
df_normalized = pd.DataFrame(scaler.fit_transform(train_df), columns=train_df.columns)

In [208]:
# Cargar DataFrame
df = df_normalized
# Separar datos de entrada y salida
Y = df[['Índice de absorción', 'Longitud de onda']]
X = df[['Fracvol', 'Radio (nm)', 'Espesor de la película (nm)']]

In [209]:
# Convertimos los dataframes en numpy arrays
X = np.array(X)
y = np.array(Y)

In [210]:
# Creamos el modelo de la red neuronal
model = Sequential()
model.add(Dense(64, input_dim=3, activation='relu'))
model.add(Dense(2, activation='linear'))

In [211]:
model.compile(loss='mean_absolute_error', optimizer='adam')

In [212]:
model.fit(X, y, epochs=10, batch_size=32)

Epoch 1/10
960/960 [==============================] - 1s 623us/step - loss: 0.1083
Epoch 2/10
960/960 [==============================] - 1s 607us/step - loss: 0.0950
Epoch 3/10
960/960 [==============================] - 1s 598us/step - loss: 0.0948
Epoch 4/10
960/960 [==============================] - 1s 602us/step - loss: 0.0947
Epoch 5/10
960/960 [==============================] - 1s 599us/step - loss: 0.0947
Epoch 6/10
960/960 [==============================] - 1s 583us/step - loss: 0.0948
Epoch 7/10
960/960 [==============================] - 1s 581us/step - loss: 0.0947
Epoch 8/10
960/960 [==============================] - 1s 593us/step - loss: 0.0948
Epoch 9/10
960/960 [==============================] - 1s 646us/step - loss: 0.0947
Epoch 10/10
960/960 [==============================] - 1s 580us/step - loss: 0.0947


In [213]:
arr = df_normalized[['Fracvol', 'Radio (nm)', 'Espesor de la película (nm)']]

In [214]:
y_pred = model.predict(arr)
print(y_pred)

960/960 [==============================] - 1s 529us/step
[[0.01163281 0.6875825 ]
 [0.02485019 0.67850894]
 [0.00553828 0.7068578 ]
 ...
 [0.00948662 0.6974232 ]
 [0.01453865 0.6846661 ]
 [0.01149192 0.6914752 ]]


# Renombrando las columnas

In [223]:
predicciones = pd.DataFrame(y_pred)
predicciones.columns = ['Índice de absorción', 'Longitud de onda']
predicciones

,Índice de absorción,Longitud de onda
0,0.011633,0.687582
1,0.024850,0.678509
2,0.005538,0.706858
3,0.009487,0.697423
4,0.011492,0.691475
...,...,...
30715,0.020802,0.686799
30716,0.007283,0.704270
30717,0.009487,0.697423
30718,0.014539,0.684666


# Probando la accurancy del modelo

Primero lo haremos por cada parámetro. Se le restará los valores predichos por la red a los valores originales de test. Se contarán el número de ceros y ese será lo acertada que es nuestra red. De tener valores inferiores a 80% en número de ceros este rango se incrementará hasta llegar a un valor que el 80% de los valores sean menores a este

# Longitud de onda

In [231]:
redondeo = predicciones['Longitud de onda'].to_numpy()
test_df_long = df_normalized['Longitud de onda'].to_numpy()
dif = redondeo - test_df_long 
#count_nonzero() cuenta el número de elementos distintos de cero en el arreglo, mientras que la función size() devuelve el tamaño total del arreglo. 
total = dif.size
num_zeros = dif.size - np.count_nonzero(dif)
porcentaje = num_zeros*100/total
print("presición del ", porcentaje, "%.")

presición del  0.0 %.


In [228]:
# Se agrega umbral hasta llegar a 80% de aceptación
i = 0
while(porcentaje < 80):
    umbral = i
    num = np.count_nonzero(np.abs(dif) <= umbral)
    porcentaje = num*100/total
    print("presición del ", porcentaje, "%. Con el redondeo para arriba en espesor y un rango de", umbral)
    i = i + .1

presición del  0.0 %. Con el redondeo para arriba en espesor y un rango de 0
presición del  29.853515625 %. Con el redondeo para arriba en espesor y un rango de 0.1
presición del  59.9609375 %. Con el redondeo para arriba en espesor y un rango de 0.2
presición del  89.990234375 %. Con el redondeo para arriba en espesor y un rango de 0.30000000000000004


In [218]:
redondeo = predicciones['Índice de absorción'].to_numpy()
test_df_long = df_normalized['Índice de absorción'].to_numpy()
dif = redondeo - test_df_long 
#count_nonzero() cuenta el número de elementos distintos de cero en el arreglo, mientras que la función size() devuelve el tamaño total del arreglo. 
total = dif.size
num_zeros = dif.size - np.count_nonzero(dif)
porcentaje = num_zeros*100/total
print("presición del ", porcentaje, "%.")

presición del  0.0 %.


In [219]:
# Se agrega umbral hasta llegar a 80% de aceptación
i = 0
while(porcentaje < 80):
    umbral = i
    num = np.count_nonzero(np.abs(dif) <= umbral)
    porcentaje = num*100/total
    print("presición del ", porcentaje, "%. Con el redondeo para arriba en espesor y un rango de", umbral)
    i = i + .01

presición del  0.0 %. Con el redondeo para arriba en espesor y un rango de 0
presición del  57.51953125 %. Con el redondeo para arriba en espesor y un rango de 0.01
presición del  76.28255208333333 %. Con el redondeo para arriba en espesor y un rango de 0.02
presición del  83.154296875 %. Con el redondeo para arriba en espesor y un rango de 0.03
